In [7]:
def make_jupyter_cells_wide():
    from IPython.display import display, HTML
    display(HTML(
        '<style>'
            '#notebook { padding-top:0px !important; } '
            '.container { width:100% !important; } '
            '.end_space { min-height:0px !important; } '
        '</style>'
    ))
print('make_jupyter_cells_wide()')
make_jupyter_cells_wide()

make_jupyter_cells_wide()


In [10]:
"""Initialize and Function setup.
"""
from typing import Dict
import datetime
import pandas as pd

def get_type(my_string, pandas=False):
    """Will return an interpreted data type of any Py variable"""
    if my_string is None:
        return None
    elif is_datetime(my_string) and pandas:
        return 'datetime64[ns]'
    elif is_datetime(my_string):
        return 'datetime'
    elif isinstance(my_string, str):
        try:
            int_val = int(my_string)
            return 'int'
        except ValueError:
            try:
                float_val = float(my_string)
                return 'float'
            except ValueError:
                return 'str'
    elif isinstance(my_string, int):
        return 'int'
    elif isinstance(my_string, float):
        return 'float'
    elif isinstance(my_string, decimal.Decimal):
        return 'decimal'
    elif isinstance(my_string, list):
        return 'list'
    elif isinstance(my_string, dict):
        return 'dict'
    elif isinstance(my_string, tuple):
        return 'tuple'
    else:
        return type(my_string)

def list_to_df(my_list, first_row_is_header=True):
    """Give me a list and I will look at the first row of data and PROPERLY coerce
    and convert it to a DataFrame with proper column data types.
    """
    if first_row_is_header:
        column_names = my_list[0]
        data = my_list[1:]
    else:
        column_names = [f'col{x}' for x in range(len(my_list[0]))]
        data = my_list
    column_types = [get_type(x, True) for x in data[0]]
    column_dict = dict(zip(column_names, column_types))
    df = pd.DataFrame(data, columns=column_names)
    for col_name, data_type in column_dict.items():
        if data_type == 'decimal':
            df[col_name] = df[col_name].astype('float')
        elif data_type == 'datetime64[ns]':
            df[col_name] = pd.to_datetime(df[col_name], errors='ignore')
        else:
            df[col_name] = df[col_name].astype(data_type)
    return df

def get_non_blanks(data: Dict[any, any]):
    """Give me a dict and I will return only the keys that have valid data"""
    non_blank_keys = []
    if data != {}:
        for key in data.keys():
            if data[key] is not None:
                if get_type(data[key]) in ['int', 'float', 'decimal']:
                    non_blank_keys.append(key)
                elif get_type(data[key]) in ['str'] and data[key] != '':
                    non_blank_keys.append(key)
                elif get_type(data[key]) not in ['str']:
                    non_blank_keys.append(key)
    return non_blank_keys


def list_to_dict(rows, column_names=None):
    """Give me a list with the first row as column names
    and I will return a dict or a list of dicts.
    Can handle rows in as a one or multi dimensional list"""
    if column_names:
        rows_to_process = rows
    else:
        column_names = rows[0]
        rows_to_process = rows[1:]
    out = []
    if len(rows_to_process) > 0:
        if not isinstance(rows_to_process[0], list):
            rows_to_process = [rows_to_process]
        for row in rows_to_process:
            data = {}
            for i, col in enumerate(column_names):
                if i+1 > len(row):
                    row.append('')
                data[col] = row[i]
            out.append(data)
    return out

def fmt_num(the_obj, format_code=","):
    if type(the_obj) is not float and type(the_obj) is not int:
        try:
            the_num = float(the_obj)
        except:
            return '0'
    else:
        the_num = the_obj
    if the_obj in ['0', '']:
        return "{0:.0f}".format(the_num)
    if format_code in ['thous', ',']:
        return "{0:,.0f}".format(the_num)
    if format_code in ['thous1', ',1', '1']:
        return "{0:,.1f}".format(the_num)
    if format_code in ['thous2', ',2', '2']:
        return "{0:,.2f}".format(the_num)
    if format_code in ['pct', '%']:
        return "{0:.0%}".format(the_num)
    if format_code in ['pct2', '%2']:
        return "{0:.2%}".format(the_num)
    if format_code in ['.1', '.']:
        return "{:.1f}".format(the_num)
    if format_code in ['.2']:
        return "{:.2f}".format(the_num)
    if format_code in ['.3']:
        return "{:.3f}".format(the_num)
    if format_code in ['pct1', '%1']:
        return "{0:.1%}".format(the_num)
    return "{0:,.0f}".format(the_num)

def convert_to_float(rows):
    """Convert all numbers in a list to Floats"""
    out_rows = []
    for row in rows:
        out_row = []
        for cell in row:
            try:
                float_cell = float(cell)
            except:
                float_cell = cell
            out_row.append(float_cell)
        out_rows.append(out_row)
    return out_rows

def is_datetime(my_string):
    # List of common date time formats to check against
    date_formats = ['%Y-%m-%d', '%m/%d/%Y', '%m-%d-%Y', '%d-%m-%Y', '%d/%m/%Y', '%Y%m%d', '%m%d%Y', '%d%m%Y']
    time_formats = ['%H:%M:%S', '%I:%M:%S %p', '%H%M%S']
    # Loop through formats and try to parse string as datetime
    if isinstance(my_string, datetime.datetime) or isinstance(my_string, datetime.date):
        return True
    if my_string is not None and isinstance(my_string, str):
        if len(my_string) >= 8:
            for date_fmt in date_formats:
                for time_format in time_formats:
                    for ty in ['date_only', 'time_only', 'both']:
                        if ty == 'date_only':
                            try:
                                dt = datetime.datetime.strptime(my_string, f'{date_fmt}')
                                return True
                            except ValueError:
                                pass
                        elif ty == 'time_only':
                            try:
                                dt = datetime.datetime.strptime(my_string, f'{time_format}')
                                return True
                            except ValueError:
                                pass
                        elif ty == 'both':
                            try:
                                dt = datetime.datetime.strptime(my_string, f'{date_fmt} {time_format}')
                                return True
                            except ValueError:
                                pass
                            try:
                                dt = datetime.datetime.strptime(my_string, f'{date_fmt}{time_format}')
                                return True
                            except ValueError:
                                pass
    return False

In [12]:
assert get_type(1) == 'int'
assert is_datetime('2025-01-01')